In [1]:
import odrive
from odrive.enums import *
import time
import math
import numpy as np
import sympy as sp
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt
plt.ion()
import matplotlib as mpl
mpl.use('Qt5Agg')
from scipy.optimize import minimize

# FK & IK

In [2]:
(thetaL_sym, thetaR_sym) = sp.symbols("""thetaL_sym thetaR_sym """, real = True)

def T(theta, x, y):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), x], 
                      [sp.sin(theta), sp.cos(theta), y],
                      [0, 0, 1]])

l1 = 0.09
l2 = 0.16
w  = 0.07

x_r = -w/2 + l1*sp.cos(thetaR_sym)
x_l = w/2 + l1*sp.cos(thetaL_sym)

y_r = l1*sp.sin(thetaR_sym)
y_l = l1*sp.sin(thetaL_sym)

theta3_sym = sp.atan2(y_l - y_r, x_l - x_r)
L = sp.sqrt((x_l - x_r)**2 + (y_l - y_r)**2)

FK = T(thetaL_sym, w/2, 0)@T((sp.pi - thetaL_sym + theta3_sym), l1, 0)@sp.Matrix([L/2, -sp.sqrt(l2**2 - (L/2)**2), 1])
FK = FK[:2,:]
FK_fast = lambdify((thetaR_sym, thetaL_sym), FK) # (x,y) = FK_fast(thetaR,thetaL)

J = FK.jacobian([thetaR_sym, thetaL_sym]).evalf()
J_fast = lambdify((thetaR_sym, thetaL_sym), J) # J_fast(thetaR,thetaL)

def IK_5_link(x, y, l1 = 0.09, l2 = 0.16, w = 0.07):

    def leg_wide(var):
        return np.linalg.norm([var[0], var[1] - np.pi]) 

    def x_constraint_equation(var):
    # should be equal to zero when the 
        return l1**2 - l2**2 + (x - w/2)**2 + y**2 - 2*l1*(y*np.sin(var[0]) + (x - w/2)*np.cos(var[0]))

    def y_constraint_equation(var):
        return l1**2 - l2**2 + (x + w/2)**2 + y**2 - 2*l1*(y*np.sin(var[1]) + (x + w/2)*np.cos(var[1]))

    res = minimize(leg_wide, (0.1, 9*np.pi/10), method="SLSQP", constraints= ({"type": "eq", "fun": x_constraint_equation}, 
                                                                              {"type": "eq", "fun": y_constraint_equation}))
    return (res.x[0], res.x[1])

## Connect to Odrive

In [4]:
def connect_odrive(serial):
    odrv0 = odrive.find_any(serial_number = serial, timeout = 20)
    if odrv0 is not None:
        print('Connected!')
        print('Target serial {} \t Odrive serial {}'.format(serial, odrv0.serial_number))
        print('Motor 0 calibrated: {}'.format(odrv0.axis0.motor.is_calibrated))
        print('Motor 1 calibrated: {}'.format(odrv0.axis1.motor.is_calibrated))    
    else:
        print("ODrive ({}) not found".format(serial))
    return odrv0

odrv1 = connect_odrive("335C315E3536")
# odrv1 = connect_odrive("206A33A5304B")

m0  = odrv1.axis0
m1  = odrv1.axis1

Connected!
Target serial 335C315E3536 	 Odrive serial 56471058265398
Motor 0 calibrated: True
Motor 1 calibrated: True


## Configuration Setup

In [5]:
def SetParameter(odrv0):
    current_limit = 20
    odrv0.axis0.motor.config.current_lim    = current_limit #A
    odrv0.axis1.motor.config.current_lim    = current_limit #A
    odrv0.axis0.controller.config.vel_limit = 1000000
    odrv0.axis1.controller.config.vel_limit = 1000000
    odrv0.axis0.controller.config.pos_gain  = 60
    odrv0.axis1.controller.config.pos_gain  = 60
    odrv0.axis0.controller.config.vel_integrator_gain = 0.0001
    odrv0.axis1.controller.config.vel_integrator_gain = 0.0001
    return(current_limit)

def CheckMotorError(odrv0):
    axis0error = odrv0.axis0.error 
    axis1error = odrv0.axis1.error 
    axis0motorerror = odrv0.axis0.motor.error 
    axis1motorerror = odrv0.axis1.motor.error 
    axis0controllererror = odrv0.axis0.controller.error 
    axis1controllererror = odrv0.axis0.controller.error 
    print("error:          ", axis0error, axis1error)
    print("motorerror:     ", axis0motorerror, axis1motorerror)
    print("controllererror:", axis0controllererror, axis1controllererror)
    return()

def MotorClearError(odrv0):
    odrv0.axis0.error = 0
    odrv0.axis1.error = 0
    odrv0.axis0.motor.error = 0
    odrv0.axis1.motor.error = 0
    odrv0.axis0.controller.error = 0
    odrv0.axis1.controller.error = 0
    odrv0.axis0.encoder.error = 0
    odrv0.axis1.encoder.error = 0
    return()

In [6]:
SetParameter(odrv1)
print("Parameter Set!")

Parameter Set!


In [7]:
print("odrive1")
CheckMotorError(odrv1)
MotorClearError(odrv1)
print("Error Clear!")

odrive1
error:           0 0
motorerror:      0 0
controllererror: 0 0
Error Clear!


## Motor Command

In [8]:
def Calibration(odrv0):
    odrv0.axis0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    odrv0.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    print('Motor 0 calibrated: {}'.format(odrv0.axis0.motor.is_calibrated))
    print('Motor 1 calibrated: {}'.format(odrv0.axis1.motor.is_calibrated))   

def ClosedLoopControlMode(odrv0):
    odrv0.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
    odrv0.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

def IdleMode(odrv0):
    odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
    odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

def ControllerPositionControl(odrv0): # default mode
    odrv0.axis0.controller.config.control_mode = CTRL_MODE_POSITION_CONTROL
    odrv0.axis1.controller.config.control_mode = CTRL_MODE_POSITION_CONTROL
#     print("Control Mode :{}".format(odrv0.axis0.controller.config.control_mode))

def ControllerCurrentControl(odrv0):
    odrv0.axis0.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL
    odrv0.axis1.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL
#     print("Control Mode :{}".format(odrv0.axis0.controller.config.control_mode))

def MaxCurrentProtection(current, current_limit = 20):
    current = min(np.abs(current), current_limit) * np.sign(current) 
    return(current)

In [ ]:
Calibration(odrv1)

In [9]:
ClosedLoopControlMode(odrv1)

In [10]:
IdleMode(odrv1)

## Define zero angle

In [11]:
m0_home_pos = m0.encoder.pos_estimate
m1_home_pos = m1.encoder.pos_estimate
theta_R_home = -((m0_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_home = -((m1_home_pos) / 8192 * 2 * np.pi)
print(m0_home_pos)
print(m1_home_pos)

-409.000732421875
1363.7530517578125


## Define home location

In [12]:
m0_equilibrium_toe_pos = m0.encoder.pos_estimate
m1_equilibrium_toe_pos = m1.encoder.pos_estimate
theta_R_eq = - ((m0_equilibrium_toe_pos - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_eq = - ((m1_equilibrium_toe_pos - m1_home_pos) / 8192 * 2 * np.pi)
theta_R_eq_d = theta_R_eq / (2 * np.pi) * 360
theta_L_eq_d = theta_L_eq / (2 * np.pi) * 360
print("Odrv1 \nR:", theta_R_eq_d, "\nL:", theta_L_eq_d)

Odrv1 
R: 139.5263671875 
L: 39.6826171875


## Space Convertor

In [13]:
def get_joint_angle(m0_home_pos, m1_home_pos):
    
    thetaR = -((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
    thetaL = -((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
    
    return np.array([[thetaR],[thetaL]])

def get_encoder_unit(m0_home_pos, m1_home_pos, theta_R, theta_L):
    
    m0_pos = -(theta_R - np.pi)/(2*np.pi)*8192 + m0_home_pos
    m1_pos = -theta_L/(2*np.pi)*8192 + m1_home_pos 
    
    return np.array([[m0_pos],[m1_pos]])

def from_joint_get_pos(m0_home_pos, m1_home_pos): #return current xy
    
    thetaR = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
    thetaL = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)

    x = FK_fast(thetaR, thetaL)[0]
    y = FK_fast(thetaR, thetaL)[1]
    
    return np.array([x, y])

def from_pos_get_joint(x, y, m0_home_pos, m1_home_pos): #xy ->encoder

    theta_R = IK_5_link(x, y)[1]
    theta_L = IK_5_link(x, y)[0]
    
    m0_pos = -(theta_R - np.pi)/(2*np.pi)*8192 + m0_home_pos
    m1_pos = -theta_L/(2*np.pi)*8192 + m1_home_pos
    
    return np.array([m0_pos, m1_pos])

## Trajactory Generator

In [14]:
def SwingTrajectory(t_clock, UpperAMP = 0.07, StanceHeight = 0.185, StepLength = 0.18):
    
    period = 2
    percent = t_clock / period

    if (percent <= 1): # Swing
        x = -(StepLength / 2) + percent * StepLength
        y = -UpperAMP * np.sin(np.pi * percent) + StanceHeight  
        
    else: # Stay and do nothing
        x = StepLength / 2
        y = StanceHeight

    return np.array([[x], [y]])

def GoHomeTrajactory(t_clock, StanceHeight = 0.185, StepLength = 0.18, LiftHeight = 0.09):

    period = 4   
    percent = t_clock / period

    if percent >= 0 and percent < 0.25:
        x = StepLength / 2
        y = -(percent / 0.25) * LiftHeight + StanceHeight
        
    elif percent >= 0.25 and percent < 0.75:
        x = StepLength / 2 - ((percent - 0.25) / 0.5) * StepLength
        y = -LiftHeight + StanceHeight
        
    elif percent >= 0.75 and percent <= 1:
        x = -StepLength / 2
        y = -LiftHeight + StanceHeight + ((percent - 0.75) / 0.25) * LiftHeight 
        
    else:
        x = -StepLength / 2
        y = StanceHeight

    return np.array([[x], [y]])

## Run test

In [15]:
%matplotlib qt

In [16]:
def run_step_test(test_cycle, pause_time = 2, swing_time = 2, gohome_time = 4):
    
    ClosedLoopControlMode(odrv1)
    
    x = []
    y = []
    x_real = []
    y_real = []
    
    count = 0
    while count < test_cycle:
        '''
        Pausing =================================================================================
        '''
        t_pause_start = time.time()
        while time.time() - t_pause_start < pause_time:

            pass
        '''
        Swing Trajactory ========================================================================
        '''
        t_swing_start = time.time()
        while time.time() - t_swing_start < swing_time:

            t_clock = time.time() - t_swing_start
            odrv1_theta_R = IK_5_link(SwingTrajectory(t_clock)[0],SwingTrajectory(t_clock)[1])[1]
            odrv1_theta_L = IK_5_link(SwingTrajectory(t_clock)[0],SwingTrajectory(t_clock)[1])[0]
            
            m0_pos = -(odrv1_theta_R - np.pi)/(2*np.pi)*8192 + m0_home_pos
            m1_pos = -odrv1_theta_L/(2*np.pi)*8192 + m1_home_pos
            
            m0.controller.set_pos_setpoint(m0_pos,0,0)
            m1.controller.set_pos_setpoint(m1_pos,0,0)
            
            #plotting-----------------------------------------------------------------------------
            x.append(SwingTrajectory(t_clock)[0])
            y.append(SwingTrajectory(t_clock)[1])
            
            real_thetaR = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
            real_thetaL = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
            
            x_real.append(FK_fast(real_thetaR, real_thetaL)[0])
            y_real.append(FK_fast(real_thetaR, real_thetaL)[1])
        '''
        Pausing =================================================================================
        '''
        ClosedLoopControlMode(odrv1)
        t_pause_start = time.time()
        while time.time() - t_pause_start < pause_time:

            pass
        '''
        Going Home Trajactory =====================================================================
        '''
        t_gohome_start = time.time()
        while time.time() - t_gohome_start < gohome_time:

            t_clock = time.time() - t_gohome_start
            odrv1_theta_R = IK_5_link(GoHomeTrajactory(t_clock)[0],GoHomeTrajactory(t_clock)[1])[1]
            odrv1_theta_L = IK_5_link(GoHomeTrajactory(t_clock)[0],GoHomeTrajactory(t_clock)[1])[0] 
            
            m0_pos = -(odrv1_theta_R - np.pi)/(2*np.pi)*8192 + m0_home_pos
            m1_pos = -odrv1_theta_L/(2*np.pi)*8192 + m1_home_pos
            
            m0.controller.set_pos_setpoint(m0_pos,0,0)
            m1.controller.set_pos_setpoint(m1_pos,0,0)
            
            #plotting-----------------------------------------------------------------------------
            x.append(GoHomeTrajactory(t_clock)[0])
            y.append(GoHomeTrajactory(t_clock)[1])    
            
            real_thetaR = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
            real_thetaL = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
            
            x_real.append(FK_fast(real_thetaR, real_thetaL)[0])
            y_real.append(FK_fast(real_thetaR, real_thetaL)[1])

        count += 1
        print("test cycle:",count)
        
        return x, y, x_real, y_real

In [17]:
t0 = time.time()
x, y, x_real, y_real = run_step_test(40)
t = time.time() - t0
print("time spent:{}sec".format(t))

IdleMode(odrv1)

test cycle: 1
test cycle: 2
test cycle: 3
test cycle: 4
test cycle: 5
test cycle: 6
test cycle: 7
test cycle: 8
test cycle: 9
test cycle: 10
test cycle: 11
test cycle: 12
test cycle: 13
test cycle: 14
test cycle: 15
test cycle: 16
test cycle: 17
test cycle: 18
test cycle: 19
test cycle: 20
test cycle: 21
test cycle: 22
test cycle: 23
test cycle: 24
test cycle: 25
test cycle: 26
test cycle: 27
test cycle: 28
test cycle: 29
test cycle: 30
test cycle: 31
test cycle: 32
test cycle: 33
test cycle: 34
test cycle: 35
test cycle: 36
test cycle: 37
test cycle: 38
test cycle: 39
test cycle: 40
time spent:400.71169424057007sec


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [ ]:
plt.plot(x,y)
plt.plot(x_real,y_real)
plt.xlim(0.1, -0.1)
plt.ylim(0.2, 0.09)
plt.title("Pos step:{} cycles & vel_gain: {}".format(test_cycle, odrv1.axis0.controller.config.vel_integrator_gain))
plt.legend(['set point','real point'])
plt.xlabel('x(m)')
plt.ylabel('y(m)')
plt.show()

## Run compliance mode

In [ ]:
ControllerCurrentControl(odrv1)

In [ ]:
"""
k = [kx,0 ]  c = [cx,0 ]
    [0 ,ky]      [0 ,cy]

"""

def compliance_swing(theta_R, theta_L, vel_R, vel_L, t_swing_start):
    
    t_clock = time.time() - t_swing_start
    disp    = FK_fast(theta_R, theta_L) - SwingTrajectory(t_clock) 
    
    Kt = 0.0285
    c  = np.array([[0.5,0],[0,0.5]])
    k  = np.array([[100 ,0],[0,100 ]])

    vel     = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    current = -J_fast(theta_R, theta_L).T @ ( k @ disp + c @ vel )*(1/Kt)
    
    return current

In [ ]:
%matplotlib qt

In [ ]:
def run_compliance_step_test(test_cycle, pause_time = 2, swing_time = 2, gohome_time = 4):
    
    ClosedLoopControlMode(odrv1)

    x = []
    y = []
    m0_i   = []
    m1_i   = []
    x_real = []
    y_real = []
    i_index = 0
    
    count = 0
    while count < test_cycle:
        '''
        Pausing =================================================================================
        '''
        t_pause_start = time.time()
        while time.time() - t_pause_start < pause_time:

            pass
        '''
        Swing Trajactory ========================================================================
        '''
        t_swing_start = time.time()
        while time.time() - t_swing_start < swing_time:
            
            t_clock = time.time() - t_swing_start
            theta_R = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
            theta_L = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
            vel_R = - (m0.encoder.vel_estimate / 8192 * 2 * np.pi)
            vel_L = - (m1.encoder.vel_estimate / 8192 * 2 * np.pi)
             
            current = compliance_swing(theta_R, theta_L, vel_R, vel_L, t_swing_start)
            # Motor current = Current - Damp Current
            m0_current_R = current[0] - np.array(0.5 * vel_R)
            m1_current_L = current[1] - np.array(0.5 * vel_L)
            
            m0.controller.set_current_setpoint(-MaxCurrentProtection(m0_current_R))
            m1.controller.set_current_setpoint(-MaxCurrentProtection(m1_current_L))
            
            #plotting-----------------------------------------------------------------------------
            x.append(SwingTrajectory(t_clock)[0])
            y.append(SwingTrajectory(t_clock)[1])
            
            real_thetaR = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
            real_thetaL = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
            
            x_real.append(FK_fast(real_thetaR, real_thetaL)[0])
            y_real.append(FK_fast(real_thetaR, real_thetaL)[1])
            
            m0_i.append(MaxCurrentProtection(m0_current_R))
            m1_i.append(MaxCurrentProtection(m1_current_L))
            i_index += 1
        '''
        Pausing =================================================================================
        '''
        m0.controller.set_current_setpoint(0)
        m1.controller.set_current_setpoint(0)
        ControllerPositionControl(odrv1)
        ClosedLoopControlMode(odrv1)
        t_pause_start = time.time()
        while time.time() - t_pause_start < pause_time:

            pass
        '''
        Going Home Trajactory =====================================================================
        '''
        t_gohome_start = time.time()
        while time.time() - t_gohome_start < gohome_time:

            t_clock = time.time() - t_gohome_start
            odrv1_theta_R = IK_5_link(GoHomeTrajactory(t_clock)[0],GoHomeTrajactory(t_clock)[1])[1]
            odrv1_theta_L = IK_5_link(GoHomeTrajactory(t_clock)[0],GoHomeTrajactory(t_clock)[1])[0] 
            
            m0_pos  = -(odrv1_theta_R - np.pi)/(2*np.pi)*8192 + m0_home_pos
            m1_pos  = -odrv1_theta_L/(2*np.pi)*8192 + m1_home_pos
            
            m0.controller.set_pos_setpoint(m0_pos,0,0)
            m1.controller.set_pos_setpoint(m1_pos,0,0)
            
            #plotting-----------------------------------------------------------------------------
            x.append(GoHomeTrajactory(t_clock)[0])
            y.append(GoHomeTrajactory(t_clock)[1])    
            
            real_thetaR = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
            real_thetaL = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
            
            x_real.append(FK_fast(real_thetaR, real_thetaL)[0])
            y_real.append(FK_fast(real_thetaR, real_thetaL)[1])

        count += 1
        print("test cycle:",count)
        
    fig1 = plt.plot(x,y)
    fig1 = plt.plot(x_real,y_real)
    plt.xlim(0.1, -0.1)
    plt.ylim(0.2, 0.08)
    plt.title("Run Compliance Step test: {} cycle".format(test_cycle))
    plt.legend(['set point','real point'])
    plt.xlabel('x(m)')
    plt.ylabel('y(m)')
    fig1 = plt.show()
    
#     i = np.arange(i_index)
#     fig2 = plt.plot(i,m0_i,'tab:red',linewidth=1)
#     fig2 = plt.plot(i,m1_i,'tab:blue',linewidth=1)
#     plt.title("Motor Current")
#     plt.legend(['m0','m1'])
#     plt.xlabel('Loop count')
#     plt.ylabel('Current (A)')
#     fig2 = plt.show()

In [ ]:
t0 = time.time()
run_compliance_step_test(40)
t = time.time() - t0
print("time spent:{}sec".format(t))

In [ ]:
IdleMode(odrv1)

In [ ]:
ClosedLoopControlMode(odrv1)

## Run compliance mode

In [ ]:
def anisotropic_swing(theta_R, theta_L, vel_R, vel_L, t_swing_start):
    
    t_clock = time.time() - t_swing_start
    disp    = FK_fast(theta_R, theta_L) - SwingTrajectory(t_clock) 
    
    Kt = 0.0285
    c  = np.array([[0.5,0],[0,0.5]])
    
    if disp[1] < -0.005:
        k = np.array([[100 ,0],[0,0 ]])    
    else:
        k = np.array([[100,0],[0,100]])
       
    vel     = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    current = -J_fast(theta_R, theta_L).T @ ( k @ disp + c @ vel )*(1/Kt)
    
    return current

In [ ]:
def run_anisotropic_compliance_step_test(test_cycle, pause_time = 2, swing_time = 2, gohome_time = 4):
    
    ClosedLoopControlMode(odrv1)

    x = []
    y = []
    m0_i   = []
    m1_i   = []
    x_real = []
    y_real = []
    i_index = 0
    
    count = 0
    while count < test_cycle:
        '''
        Pausing =================================================================================
        '''
        t_pause_start = time.time()
        while time.time() - t_pause_start < pause_time:

            pass
        '''
        Swing Trajactory ========================================================================
        '''
        t_swing_start = time.time()
        while time.time() - t_swing_start < swing_time:
            
            t_clock = time.time() - t_swing_start
            theta_R = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
            theta_L = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
            vel_R = - (m0.encoder.vel_estimate / 8192 * 2 * np.pi)
            vel_L = - (m1.encoder.vel_estimate / 8192 * 2 * np.pi)
             
            current = anisotropic_swing(theta_R, theta_L, vel_R, vel_L, t_swing_start)
            # Motor current = Current - Damp Current
            m0_current_R = current[0] - np.array(0.5 * vel_R)
            m1_current_L = current[1] - np.array(0.5 * vel_L)
            
            m0.controller.set_current_setpoint(-MaxCurrentProtection(m0_current_R))
            m1.controller.set_current_setpoint(-MaxCurrentProtection(m1_current_L))
            
            #plotting-----------------------------------------------------------------------------
            x.append(SwingTrajectory(t_clock)[0])
            y.append(SwingTrajectory(t_clock)[1])
            
            real_thetaR = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
            real_thetaL = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
            
            x_real.append(FK_fast(real_thetaR, real_thetaL)[0])
            y_real.append(FK_fast(real_thetaR, real_thetaL)[1])
            
            m0_i.append(MaxCurrentProtection(m0_current_R))
            m1_i.append(MaxCurrentProtection(m1_current_L))
            i_index += 1
        '''
        Pausing =================================================================================
        '''
        m0.controller.set_current_setpoint(0)
        m1.controller.set_current_setpoint(0)
        ControllerPositionControl(odrv1)
        ClosedLoopControlMode(odrv1)
        t_pause_start = time.time()
        while time.time() - t_pause_start < pause_time:

            pass
        '''
        Going Home Trajactory =====================================================================
        '''
        t_gohome_start = time.time()
        while time.time() - t_gohome_start < gohome_time:

            t_clock = time.time() - t_gohome_start
            odrv1_theta_R = IK_5_link(GoHomeTrajactory(t_clock)[0],GoHomeTrajactory(t_clock)[1])[1]
            odrv1_theta_L = IK_5_link(GoHomeTrajactory(t_clock)[0],GoHomeTrajactory(t_clock)[1])[0] 
            
            m0_pos  = -(odrv1_theta_R - np.pi)/(2*np.pi)*8192 + m0_home_pos
            m1_pos  = -odrv1_theta_L/(2*np.pi)*8192 + m1_home_pos
            
            m0.controller.set_pos_setpoint(m0_pos,0,0)
            m1.controller.set_pos_setpoint(m1_pos,0,0)
            
            #plotting-----------------------------------------------------------------------------
            x.append(GoHomeTrajactory(t_clock)[0])
            y.append(GoHomeTrajactory(t_clock)[1])    
            
            real_thetaR = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
            real_thetaL = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
            
            x_real.append(FK_fast(real_thetaR, real_thetaL)[0])
            y_real.append(FK_fast(real_thetaR, real_thetaL)[1])

        count += 1
        print("test cycle:",count)
        
    fig1 = plt.plot(x,y)
    fig1 = plt.plot(x_real,y_real)
    plt.xlim(0.1, -0.1)
    plt.ylim(0.2, 0.08)
    plt.title("Run Compliance Step test: {} cycle".format(test_cycle))
    plt.legend(['set point','real point'])
    plt.xlabel('x(m)')
    plt.ylabel('y(m)')
    fig1 = plt.show()
    
#     i = np.arange(i_index)
#     fig2 = plt.plot(i,m0_i,'tab:red',linewidth=1)
#     fig2 = plt.plot(i,m1_i,'tab:blue',linewidth=1)
#     plt.title("Motor Current")
#     plt.legend(['m0','m1'])
#     plt.xlabel('Loop count')
#     plt.ylabel('Current (A)')
#     fig2 = plt.show()

In [ ]:
t0 = time.time()
run_anisotropic_compliance_step_test(40)
t = time.time() - t0
print("time spent:{}sec".format(t))

In [ ]:
IdleMode(odrv1)

In [ ]:
ClosedLoopControlMode(odrv1)

In [ ]:
%matplotlib qt

In [ ]:
print("odrive1")
CheckMotorError(odrv1)
MotorClearError(odrv1)
print("Error Clear!")